In [1]:
# 21:30 07.05 4 hour work. Пора с этим апи писать работающую тему, показывать Афонину, чтобы он дальше направлял.
!pip install g4f

In [2]:
# 20:00. 5,5 hour work 06.05. Что я планирую сделать?
# 1). В json формат засунуть ответы gpt , пройтись по 5-10 сайтам, возможно со сменой промптов
# проверить что всё работает или найти случаи, когда может не работать и разобраться как это отлавливать.

# 2). дальше скинуть ему скрины(не стесняйся писать!!! я же по факту пишу, а не флужу) и сказать, что вот обработал 5-10 сайтов, вот такие результаты , вроде отличные.
# еще могу сказать ему, что если надо или интересно ему, то могу еще прикрутить тг бота, который по ссылке и промпту вернет всё пользователю
# но приходится долго ждать, поскольку мы дергаем апи и зафайнтюнить тут очев не выйдет
# 3). следующий этап тогда скачать ллм уже локально (лучше на мак, он точно не сломается) , проверить как они будут работать
# если что-то нормальное, то отлично идём файнтюнить (hf Trainer or AutoTrain) читаю доку, там должно быть очень просто по словам Влада и делаю
# 4). в конце концов обсудить с ним(именно с ним, а не с друзьями-кентами, у которых могут быть другие требования)
# что мне нужно писать в самой курсовой, есть ли какой-то жеский формат или я делаю презентацию просто в свободной форме

In [14]:
test_conferences = [
    "https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html",
    "https://icerm.brown.edu/topical_workshops/tw-24-ips/",
    "https://www.siam.org/conferences/cm/conference/la24",
    "https://www.birs.ca/events/2024/5-day-workshops/24w5313",
    "https://icerm.brown.edu/topical_workshops/tw-24-saso/",
    "https://indico.ictp.it/event/10516",
    "https://icofca.com/",
    "https://moad2024.sciencesconf.org/", # нет спикеров
    "http://caim.romai.ro/Caim24/caim_24.htm", # нет спикеров
    "https://ima.org.uk/23503/4th-ima-conference-on-inverse-problems-from-theory-to-application/", # обрати внимание, чтобы извлекались только спикеры, а не организационный комитет
    "https://sites.google.com/view/atsf2024/home?authuser=0", # нет спикеров, попробуй отдельно отсюда их извлечь https://sites.google.com/view/atsf2024/invited-speakers?authuser=0
    "https://boolean.w.uib.no/bfa-2024/", # опять же проверь, что только нужные спикеры извлекаются, а не орг комитеты еще и тд
    ]

In [15]:
from g4f.client import Client

client = Client()
def ask(promt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": promt}],
    )

    return response.choices[0].message.content

user_agent = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0"
headers = {"User-Agent": user_agent}

In [16]:
import numpy as np
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

def extract_information(conferences):
    texts = {url: "" for url in conferences}
    resps = {url: "" for url in conferences}
    MAXLEN = 15000
    sleep_time = [1, 2, 3, 4, 5]

    for url in tqdm(conferences):
        try:
            request = Request(url, None, headers)
            response = urlopen(request)
        except HTTPError as err:
            texts[url] = None
            resps[url] = "Sorry, I can't follow this link"
            continue

        html = response.read()
        soup = BeautifulSoup(html, features="html.parser")

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        text = text[:MAXLEN]

        texts[url] = text
        promt = 'find and return 4 lists (in structure: (number of list) : [only data in list, without any artifacts]): first includes only: "topics", second  includes only: "speakers", third includes only: "time and date",  fourth includes only:"place" ' + text
        resps[url] = ask(promt)

        sleep(np.random.choice(sleep_time)) # not banned

    return texts, resps

In [17]:
texts, resps = extract_information(test_conferences[:7])

100%|██████████| 7/7 [00:45<00:00,  6.56s/it]


In [7]:
def find_date(dates):
    result = []
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    nums = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for date in dates:
        if sum(1 if month in date.lower() else 0 for month in months) * sum(1 if num in date.lower() else 0 for num in nums):
            result.append(date)

    return result

In [8]:
import re

def re_preprocessing(resp, url):
    if re.search(r'[a-z]', resp.lower()) is None:
        return {"gpt_resp": resp, "warning": "Sorry I couldn't process this site. Please try again"}

    keys = ["topics", "speakers", "time and date", "place"]
    indexes = dict()
    for i, key in enumerate(keys):
        s = re.search(f'{i + 1}' + r'.{0,}' + f'{key}', resp.lower())
        indexes[key] = resp.lower().find(s[0]) if s else -1

    res = {"url": url}
    res["gpt_resp"] = resp
    # нет ключевых слов в выдаче, значит gpt вернула ответ в формате списков
    if set(indexes.values()) == {-1}:
        for i, key in enumerate(keys):
            if "\n\n" in resp:
                theme = resp.split("\n\n")[i]
            else:
                theme = resp.split("\n")[i]
            theme = theme[theme.find('['):]
            try:
                theme = eval(theme)
            except:
                theme = theme[1:-1].split(", ")

            res[key] = theme if len(theme) > 0 and re.search("\w", f"{theme[0]}") else [f"There's no specific list of {key} provided in the text"]

        res["time and date"] = find_date(res["time and date"])
        return res

    for i, key in enumerate(keys):
        if i + 1 == len(keys):
            theme = resp[indexes[key]:]
            theme = theme if "\n\n" not in theme else theme[:theme.find("\n\n")]
        else:
            theme = resp[indexes[key]:indexes[keys[i + 1]]]

        theme = theme.strip()
        if "\n" in theme: # то есть разделены по переносу строки
            theme = [t.strip() for t in theme.split("\n")[1:] if t]
            theme = [t[1:].strip() if t.startswith("-") else t for t in theme]
        else: # вероятно ответ оформлен в виде списка с перечислением через запятую: '[a, b, c, d]' or '["a", "b", "c", "d"]'
            theme = theme[theme.find('['):]
            try:
                theme = eval(theme)
            except:
                theme = theme[1:-1].split(", ")

        theme = theme[1:] if key.lower() in theme[0].lower() else theme
        res[key] = theme if len(theme) > 0 and re.search("\w", f"{theme[0]}") else [f"There's no specific list of {key} provided in the text"]

    res["time and date"] = find_date(res["time and date"])
    return res

In [18]:
resps[test_conferences[0]], len(texts[test_conferences[0]])

('', 15000)

In [19]:
resps[test_conferences[1]], len(texts[test_conferences[1]])

('', 15000)

In [20]:
resps[test_conferences[2]], len(texts[test_conferences[2]])

('当前地区当日额度已消耗完, 请尝试更换网络环境', 11577)

In [21]:
resps[test_conferences[3]], len(texts[test_conferences[3]])

('', 3819)

In [22]:
resps[test_conferences[4]], len(texts[test_conferences[4]])

('', 11602)

In [23]:
resps[test_conferences[5]], len(texts[test_conferences[5]])

('当前地区当日额度已消耗完, 请尝试更换网络环境', 3546)

In [24]:
resps[test_conferences[6]], len(texts[test_conferences[6]])

('', 7187)

In [ ]:
re_preprocessing(resps[test_conferences[7]], test_conferences[7])

{'url': 'https://icofca.com/',
 'gpt_resp': '1: ["topics", "Fractional Differential Equations", "Fractional Partial Differential Equations", "Theory of existence and uniqueness of solutions", "Stability analysis", "Boundary value problems", "Inverse problems", "Fractional Control Systems", "Applications in Physics, Engineering, Biology, and more."]\n2: ["speakers", "Prof. Dumitru Baleanu"]\n3: ["time and date", "From December 26, 2024, to December 30, 2024"]\n4: ["place", "Sousse - Tunisie"]',
 'topics': ['Fractional Differential Equations',
  'Fractional Partial Differential Equations',
  'Theory of existence and uniqueness of solutions',
  'Stability analysis',
  'Boundary value problems',
  'Inverse problems',
  'Fractional Control Systems',
  'Applications in Physics, Engineering, Biology, and more.'],
 'speakers': ['Prof. Dumitru Baleanu'],
 'time and date': ['From December 26, 2024, to December 30, 2024'],
 'place': ['Sousse - Tunisie']}

In [13]:
import json
for url in test_conferences[:1]:
    with open(f'{url.split("/")[-2]}.json', 'w') as fp:
        json.dump(re_preprocessing(resps[url], url), fp)

In [ ]:
for key, val in re_preprocessing(resps[test_conferences[0]]).items():
    print(key)
    print(val)
    print("\n\n")

gpt_resp
Here are the requested lists:

1. **Topics**:
   - Grammars, acceptors, and transducers for words, trees, and graphs
   - Algebraic theories of automata
   - Algorithmic, combinatorial, and algebraic properties of words and languages
   - Variable length codes
   - Symbolic dynamics
   - Cellular automata
   - Polyominoes and multidimensional patterns
   - Decidability questions
   - Image manipulation and compression
   - Efficient text algorithms
   - Relationships to cryptography, concurrency, complexity theory, and logic
   - Bio-inspired computing
   - Quantum computing

2. **Speakers**:
   - Tomohiro I
   - Bakhadyr Khoussainov
   - Alexander Okhotin
   - Dominique Perrin
   - Marinella Sciortino
   - Andrew Winslow

3. **Time and Date**:
   - Conference: September 10-14, 2018

4. **Place**:
   - Tokyo, Japan



topics
['Grammars, acceptors, and transducers for words, trees, and graphs', 'Algebraic theories of automata', 'Algorithmic, combinatorial, and algebraic propert

In [ ]:
print(resps[test_conferences[11]])

Here are the four lists you requested:

1. List of topics:
- Foundational theory of Boolean functions and discrete structures
- Design, proposal, and analysis of cryptographically significant (vectorial) Boolean functions
- The study of applications of (vectorial) Boolean functions to cryptography, codes, and other discrete structures
- Theory and construction of quantum Boolean functions
- Theory of finite fields and its applications in cryptography and coding theory

2. List of speakers:
- Markus Grassl, University Gdansk
- Sylvain Guilley, TELECOM-ParisTech
- Tor Helleseth, University of Bergen, Norway
- Gregor Leander, Ruhr-Universität Bochum, Germany
- Gábor Péter Nagy, Budapest University of Technology and Economics, Hungary
- Claude Carlet, University of Paris VIII, France, and University of Bergen, Norway
- Sihem Mesnager, University of Paris VIII and Sorbonne University North, France
- Stjepan Picek, Radboud University, the Netherlands

3. List of time and date:
- Submission d

In [ ]:
for key, val in re_preprocessing(resps[test_conferences[11]]).items():
    print(key)
    print(val)
    print("\n\n")

gpt_resp
Here are the four lists you requested:

1. List of topics:
- Foundational theory of Boolean functions and discrete structures
- Design, proposal, and analysis of cryptographically significant (vectorial) Boolean functions
- The study of applications of (vectorial) Boolean functions to cryptography, codes, and other discrete structures
- Theory and construction of quantum Boolean functions
- Theory of finite fields and its applications in cryptography and coding theory

2. List of speakers:
- Markus Grassl, University Gdansk
- Sylvain Guilley, TELECOM-ParisTech
- Tor Helleseth, University of Bergen, Norway
- Gregor Leander, Ruhr-Universität Bochum, Germany
- Gábor Péter Nagy, Budapest University of Technology and Economics, Hungary
- Claude Carlet, University of Paris VIII, France, and University of Bergen, Norway
- Sihem Mesnager, University of Paris VIII and Sorbonne University North, France
- Stjepan Picek, Radboud University, the Netherlands

3. List of time and date:
- Sub

In [ ]:
print(resps[test_conferences[9]])
# хороший пример когда gpt ничего не ответила

In [ ]:
# хороший пример когда time and date и place содержит несколько элементов в списке и нужно выбирать.
# в time and date у меня есть идея как , а вот на place походу забить придется и так выдавать
for key, val in re_preprocessing(resps[test_conferences[8]]).items():
    print(key)
    print(val)
    print("\n\n")

gpt_resp
Here are the four lists extracted from the provided text:

1. **Topics**:
   - Partial Differential Equations
   - Ordinary Differential Equations, Dynamical Systems
   - Mathematical Modeling in Engineering and Life Sciences
   - Real, Complex, Functional and Numerical Analysis
   - Probability Theory, Mathematical Statistics, Operations Research
   - Algebra, Logic, Geometry
   - Computer Science
   - Education

2. **Speakers**: *(No specific list of speakers is provided in the text.)*

3. **Time and Date**:
   - Conference Dates: September 19-22, 2024
   - Important Dates:
     - Deadline for submitting abstracts: August 15, 2024
     - Notification of accepted abstracts: August 20, 2024
     - Deadline for submitting final paper (didactic Section): August 20, 2024
     - Deadline for reduced fee: September 1, 2024
     - Registration deadline: September 1, 2024

4. **Place**:
   - Location: University of Oradea, Universitatii 1, Romania
   - Accommodation: Limited rooms in

In [ ]:
re_preprocessing(resps[test_conferences[3]])

{'topics': ['Homological Perspective on Splines and Finite Elements'],
 'speakers': ['Tanya Sorokina (Towson University)',
  'Martina Lanini (University of Rome Tor Vergata)',
  'Ulrich Reif (Technische Universtität Darmstadt)',
  'Deepesh Toshniwal (Delft University of Technology)',
  'Jenna Rajchgot (McMaster University)'],
 'time and date': ['May 19 - 24, 2024'],
 'place': ['UBC Okanagan campus in Kelowna, B.C., Canad']}

In [ ]:
"""
Что буду делать в среду 08.05. И если тчо не стесняйся потом еще ему писать в праздники, пох, надо работать, он точно на это не обидется:
Пробежавшись по первому тестовому сету увидел пока одну проблему: вместо спикеров любит извлечь комитет, или просто какого-то чела, особенно если спикеры не указаны
Нужно написать проверку полноты, запустить на новом тестовом сете и отправлять завтра (в среду) результаты Афонину и спрашивать, что можно делать дальше с этим
Хотя учитывая, что gpt каждый раз по-разному генерит, то в целом один и тот же сет, можно использовать постоянно, но всё-таки если не лень, то потом новый собрать
"""

In [ ]:
for url, resp in resps.items():
    print(url)
    for key, val in re_preprocessing(resp).items():
        print(key)
        print(val)
        print("\n")
    print("\n\n")

http://dlt2018.uec.ac.jp/


TypeError: re_preprocessing() missing 1 required positional argument: 'url'

In [ ]:
"3" in resps[test_conferences[11]] # обработать такие случаи

False

In [ ]:
"""
не являются спикерами
1. **Speakers**:
    - Organizing Committee Co-chairs:
        - Laura Grigori, EPFL and PSI, Switzerland
        - Daniel Kressner, EPFL, Switzerland

интересный ответ

2. **Speakers**:
- No speakers listed in the provided text.

нужно отпарсить даты и выбрать самую старшую позднюю, очев она и будет верной как дата конфференции
3: ["Submission deadline: May 15, 2024", "Acceptance notification: July 1, 2024", "Camera-ready submission: August 1, 2024", "Early registration deadline for physical participation: July 15, 2024", "Registration deadline for physical participation: August 17, 2024", "Video submission of presentations: August 20, 2024", "BFA 2024: September 09 – September 13, 2024"],
"""

'\nне являются спикерами\n1. **Speakers**:\n    - Organizing Committee Co-chairs:\n        - Laura Grigori, EPFL and PSI, Switzerland\n        - Daniel Kressner, EPFL, Switzerland\n\nинтересный ответ\n\n2. **Speakers**:\n- No speakers listed in the provided text.\n\nнужно отпарсить даты и выбрать самую старшую позднюю, очев она и будет верной как дата конфференции\n3: ["Submission deadline: May 15, 2024", "Acceptance notification: July 1, 2024", "Camera-ready submission: August 1, 2024", "Early registration deadline for physical participation: July 15, 2024", "Registration deadline for physical participation: August 17, 2024", "Video submission of presentations: August 20, 2024", "BFA 2024: September 09 – September 13, 2024"],\n'

In [ ]:
# проверка на то, если дали сайт, который вообще не относится ни к какой конференции
for key, text in texts.items():
    triggers = ["conference", "topic", "speaker", "date", "universit", "committee"]
    if sum(0 if trigger not in text.lower() else 1 for trigger in triggers) == 0:
        print(key)